In [1]:
!pip install tensorflow
!pip install -U imbalanced-learn

Requirement already up-to-date: imbalanced-learn in c:\programdata\anaconda3\lib\site-packages (0.5.0)


In [15]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.patches as mpatches
import time

# Classifier Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import collections


# Other Libraries
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold
import warnings

In [16]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, RobustScaler

from sklearn.preprocessing import LabelEncoder

In [17]:
dt = pd.read_csv('../train.csv' , encoding='big5')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (17,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
def data_cl(dt):
    dt = dt.drop("CUS_ID" ,axis=1)
    
    ohi = pd.get_dummies(dt["GENDER"])
    dt = dt.drop("GENDER",axis=1)
    dt = pd.concat([ohi,dt],axis=1)
    
    ohi = pd.get_dummies(dt["AGE"])
    dt = dt.drop("AGE",axis=1)
    dt = pd.concat([ohi,dt],axis=1)
    
    
    
#     sizemap = {
#         "低":1,
#         "中":3,
#         "中高":5,
#         "高":7
#     }

#     dt["AGE"] = dt["AGE"].map(sizemap)
    
#     for i in dt.keys():
#         if dt[i].isnull().sum()>60000:
#             dt = dt.drop(i,axis=1)

    

    labelencoder = LabelEncoder()
    for i in dt.keys():    
        for a in dt[i]:
            if  a is not None and isinstance(a,str):
                dt[i] = labelencoder.fit_transform(dt[i].astype(str))
                break
    
    for i in dt.keys():
        dt[i] = dt[i].fillna(dt[i].mean())
    
    return dt    
    

In [19]:
dt = data_cl(dt)
dt.shape

(100000, 135)

In [7]:
dt = dt.sample(frac = 1)

fraud_dt = dt.loc[dt['Y1'] == 1]
non_fraud_dt = dt.loc[dt['Y1'] == 0][:2000]

normal_distributed_dt = pd.concat([fraud_dt, non_fraud_dt])

new_dt = normal_distributed_dt.sample(frac=1)
# new_dt = normal_distributed_dt.sample(frac=1, random_state=42)

new_dt.shape

(0, 132)

In [68]:
from sklearn import  ensemble, preprocessing, metrics
from sklearn.model_selection import train_test_split

x = new_dt.drop("Y1",axis=1)
y = new_dt["Y1"]
train_X, test_X, train_y, test_y = train_test_split(x, y, test_size = 0.3)

forest = ensemble.RandomForestClassifier(n_estimators = 131)
forest_fit = forest.fit(train_X, train_y)

In [69]:
test_y_predicted = forest.predict(test_X)

# 績效
fpr, tpr, thresholds = metrics.roc_curve(test_y, test_y_predicted)
auc = metrics.auc(fpr, tpr)
print(auc)

0.7498478824402157


In [70]:
x1 = dt.drop("Y1",axis=1)
y1 = dt["Y1"]
test_y_predicted = forest.predict(x1)

# 績效
fpr, tpr, thresholds = metrics.roc_curve(y1, test_y_predicted)
auc = metrics.auc(fpr, tpr)
print(auc)

0.8256938775510203


# 預測

In [41]:
df = pd.read_csv('../test.csv' , encoding='big5')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (17,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [42]:
df = data_cl(df)
df = df.drop(x,axis=1)
df.shape

(150000, 84)

In [45]:
xt1 = df
# yt1 = df["Y1"]
test_y_predicted = forest.predict(xt1)

In [46]:
test_y_predicted

array([1, 0, 0, ..., 1, 1, 1])

In [44]:
gt1 = df




In [40]:
test_y_predicted = model.predict_classes(gt1)

NameError: name 'model' is not defined

In [121]:
test_y_predicted

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [47]:
fin = pd.read_csv('../submit_test.csv' , encoding='big5')

In [48]:
fin["Ypred"] = test_y_predicted

In [49]:
fin.to_csv('submit_test.csv')

In [20]:
X = dt.drop("Y1",axis=1)
y = dt["Y1"]

In [21]:
X.shape

(100000, 134)

In [22]:
from keras import models
from keras import layers

def build_model():
    model = models.Sequential()
    model.add(layers.Dropout(0.2 ,input_shape=(134,)))
    model.add(layers.Dense(134, activation='relu',input_shape=(X.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(32, activation="relu"))
    model.add(layers.Dense(1))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

In [126]:
model = build_model()

model.fit(X, y, epochs=500, verbose=14)

Epoch 1/500
Epoch 2/500
Epoch 3/500
Epoch 4/500
Epoch 5/500
Epoch 6/500
Epoch 7/500
Epoch 8/500
Epoch 9/500
Epoch 10/500
Epoch 11/500
Epoch 12/500
Epoch 13/500
Epoch 14/500
Epoch 15/500
Epoch 16/500
Epoch 17/500
Epoch 18/500
Epoch 19/500
Epoch 20/500
Epoch 21/500
Epoch 22/500
Epoch 23/500
Epoch 24/500
Epoch 25/500
Epoch 26/500
Epoch 27/500
Epoch 28/500
Epoch 29/500
Epoch 30/500
Epoch 31/500
Epoch 32/500
Epoch 33/500
Epoch 34/500
Epoch 35/500
Epoch 36/500
Epoch 37/500
Epoch 38/500
Epoch 39/500
Epoch 40/500
Epoch 41/500
Epoch 42/500
Epoch 43/500
Epoch 44/500
Epoch 45/500
Epoch 46/500
Epoch 47/500
Epoch 48/500
Epoch 49/500
Epoch 50/500
Epoch 51/500
Epoch 52/500
Epoch 53/500
Epoch 54/500
Epoch 55/500
Epoch 56/500
Epoch 57/500
Epoch 58/500
Epoch 59/500
Epoch 60/500
Epoch 61/500
Epoch 62/500
Epoch 63/500
Epoch 64/500
Epoch 65/500
Epoch 66/500
Epoch 67/500
Epoch 68/500
Epoch 69/500
Epoch 70/500
Epoch 71/500
Epoch 72/500
Epoch 73/500
Epoch 74/500
Epoch 75/500
Epoch 76/500
Epoch 77/500
Epoch 78

In [127]:
from sklearn import  ensemble, preprocessing, metrics
ttx = dt.drop("Y1",axis=1)
tty = dt["Y1"]
test_y_predicted = model.predict_classes(ttx)

# 績效
fpr, tpr, thresholds = metrics.roc_curve(tty, test_y_predicted)
auc = metrics.auc(fpr, tpr)
print(auc)

0.5887244897959183


In [128]:
ttx2 = new_dt.drop("Y1",axis=1)
tty2 = new_dt["Y1"]
test_y_predicted = model.predict_classes(ttx2)

# 績效
fpr, tpr, thresholds = metrics.roc_curve(tty2, test_y_predicted)
auc = metrics.auc(fpr, tpr)
print(auc)

0.58875


In [23]:
x = dt.drop("Y1",axis=1)
y = dt["Y1"]
train_X, test_X, train_y, test_y = train_test_split(x, y, test_size = 0.3)

from sklearn.linear_model  import LogisticRegression
lr=LogisticRegression()
lr.fit(train_X,train_y)


# 印出截距
print(lr.intercept_ )

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[-3.91807963]


In [24]:
# 印出係數
a = abs(lr.coef_)[0]
c =lr.coef_[0]

In [25]:
a = list(a)
c = list(c)

In [26]:
newa = list(c)
newa.sort()

In [27]:
newa

[-1.9291776089693016,
 -1.2471456723626801,
 -1.2089010998617806,
 -0.8059760425395852,
 -0.7899188654544171,
 -0.7404898037727388,
 -0.6609309136885553,
 -0.6269095000950553,
 -0.6241872555935166,
 -0.5519149923552565,
 -0.5510030621709722,
 -0.44629422944389435,
 -0.444043036979526,
 -0.3753946685270404,
 -0.3440845115231613,
 -0.32800238887123395,
 -0.29546212168525865,
 -0.2747758281258532,
 -0.2699845249938068,
 -0.2511940150691464,
 -0.23262172805645123,
 -0.23261772630309155,
 -0.21004996233498213,
 -0.19133572269555513,
 -0.18521660851466054,
 -0.1842765089129195,
 -0.1790767037425482,
 -0.1784007823779368,
 -0.16815868760753158,
 -0.1554231416169179,
 -0.15387424836062585,
 -0.15052506050334316,
 -0.13562430095035233,
 -0.13455143089409272,
 -0.1338273040967346,
 -0.13080424435401145,
 -0.1293817246378183,
 -0.12850711884286706,
 -0.1151421957560166,
 -0.11097394986417583,
 -0.11081244867956253,
 -0.0976037340625755,
 -0.09755446556203608,
 -0.0959530750438256,
 -0.07513112122

In [ ]:
dt.columns[c[c.index(newa[-2])]

In [ ]:
c[c.index(newa[-2])]

In [ ]:
dt.head()

In [28]:
x = []
for i in newa:
    if abs(i) < 0.1:
        x.append(dt.columns[c.index(i)])

In [30]:
fix_dt = dt.drop(x,axis=1)

In [31]:
fx = fix_dt.drop("Y1",axis=1)
fy = fix_dt["Y1"]

In [32]:
fix_dt = fix_dt.sample(frac = 1)

fraud_dt = fix_dt.loc[dt['Y1'] == 1]
non_fraud_dt = fix_dt.loc[dt['Y1'] == 0][:2000]

normal_distributed_dt = pd.concat([fraud_dt, non_fraud_dt])

new_dtf = normal_distributed_dt.sample(frac=1)
# new_dt = normal_distributed_dt.sample(frac=1, random_state=42)

new_dtf.shape

(4000, 85)

In [33]:
from sklearn import  ensemble, preprocessing, metrics
from sklearn.model_selection import train_test_split

xf = new_dtf.drop("Y1",axis=1)
yf = new_dtf["Y1"]
train_x, test_x, train_y, test_y = train_test_split(xf, yf, test_size = 0.3)

forest = ensemble.RandomForestClassifier(n_estimators = 131)
forest_fit = forest.fit(train_x, train_y)

In [34]:
test_y_predicted = forest.predict(test_x)

# 績效
fpr, tpr, thresholds = metrics.roc_curve(test_y, test_y_predicted)
auc = metrics.auc(fpr, tpr)
print(auc)

0.7065437823913473


In [35]:
x1 = fix_dt.drop("Y1",axis=1)
y1 = fix_dt["Y1"]
test_y_predicted = forest.predict(x1)

# 績效
fpr, tpr, thresholds = metrics.roc_curve(y1, test_y_predicted)
auc = metrics.auc(fpr, tpr)
print(auc)

0.8093316326530613
